In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to SQLite (in-memory database)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [3]:

# Create customers table
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT NOT NULL
)
''')

In [4]:
# Create orders table
cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    order_date TEXT NOT NULL,
    customer_id INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
)
''')

In [5]:

# Insert data into customers table
customers_data = [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie')
]
cursor.executemany('INSERT INTO customers (customer_id, customer_name) VALUES (?, ?)', customers_data)

In [6]:
# Insert data into orders table
orders_data = [
    (1, '2023-06-01', 1),
    (2, '2023-06-02', 1),
    (3, '2023-06-03', 2)
]
cursor.executemany('INSERT INTO orders (order_id, order_date, customer_id) VALUES (?, ?, ?)', orders_data)

conn.commit()

In [7]:
# Perform an inner join
query = '''
SELECT customers.customer_id, customers.customer_name, orders.order_id, orders.order_date
FROM customers
INNER JOIN orders ON customers.customer_id = orders.customer_id
'''
result = pd.read_sql_query(query, conn)
result


,customer_id,customer_name,order_id,order_date
0,1,Alice,1,2023-06-01
1,1,Alice,2,2023-06-02
2,2,Bob,3,2023-06-03


In [8]:
# Perform a left join
query = '''
SELECT customers.customer_id, customers.customer_name, orders.order_id, orders.order_date
FROM customers
LEFT JOIN orders ON customers.customer_id = orders.customer_id
'''
result = pd.read_sql_query(query, conn)
result


,customer_id,customer_name,order_id,order_date
0,1,Alice,1.0,2023-06-01
1,1,Alice,2.0,2023-06-02
2,2,Bob,3.0,2023-06-03
3,3,Charlie,NaN,None


In [9]:
# Perform a right join (simulated with a left join)
query = '''
SELECT orders.order_id, orders.order_date, customers.customer_id, customers.customer_name
FROM orders
LEFT JOIN customers ON orders.customer_id = customers.customer_id
'''
result = pd.read_sql_query(query, conn)
result


,order_id,order_date,customer_id,customer_name
0,1,2023-06-01,1,Alice
1,2,2023-06-02,1,Alice
2,3,2023-06-03,2,Bob


In [10]:
# Perform a full outer join (simulated)
query = '''
SELECT customers.customer_id, customers.customer_name, orders.order_id, orders.order_date
FROM customers
LEFT JOIN orders ON customers.customer_id = orders.customer_id
UNION
SELECT customers.customer_id, customers.customer_name, orders.order_id, orders.order_date
FROM orders
LEFT JOIN customers ON orders.customer_id = customers.customer_id
'''
result = pd.read_sql_query(query, conn)
result


,customer_id,customer_name,order_id,order_date
0,1,Alice,1.0,2023-06-01
1,1,Alice,2.0,2023-06-02
2,2,Bob,3.0,2023-06-03
3,3,Charlie,NaN,None


In [11]:
# Perform a cross join
query = '''
SELECT customers.customer_id, customers.customer_name, orders.order_id, orders.order_date
FROM customers
CROSS JOIN orders
'''
result = pd.read_sql_query(query, conn)
result


,customer_id,customer_name,order_id,order_date
0,1,Alice,1,2023-06-01
1,1,Alice,2,2023-06-02
2,1,Alice,3,2023-06-03
3,2,Bob,1,2023-06-01
4,2,Bob,2,2023-06-02
5,2,Bob,3,2023-06-03
6,3,Charlie,1,2023-06-01
7,3,Charlie,2,2023-06-02
8,3,Charlie,3,2023-06-03
